### Pip



In [ ]:
!pip install optuna
!pip install catboost

### Import


In [ ]:
# 전처리를 위한 라이브러리
import pandas as pd
import numpy as np
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split


In [ ]:
# 모델에 필요한 라이브러리
import lightgbm as lgb
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


### Data Load


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/LG/train.csv').drop(columns=['ID'])
test = pd.read_csv('/content/drive/MyDrive/LG/test.csv').drop(columns=['ID'])

In [ ]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing


In [ ]:
# "불임 원인 - 여성 요인"과 "난자 채취 경과일"의 값 분포 확인
X[['불임 원인 - 여성 요인', '난자 채취 경과일']].value_counts()


,,count
불임 원인 - 여성 요인,난자 채취 경과일,
0,0.0,198863


In [ ]:
## 2개 피쳐변수 Drop
## Drop 이유: 값이 모두 0으로 1개의 값으로 되어 있음
X.drop([ "불임 원인 - 여성 요인","난자 채취 경과일"], axis=1, inplace=True)
test.drop([ "불임 원인 - 여성 요인","난자 채취 경과일"], axis=1, inplace=True)

In [ ]:
# 연령대 컬럼 변환
age_mapping = {'만18-34세': 0, '만35-37세': 1, '만38-39세': 2, '만40-42세': 3, '만43-44세': 4, '만45-50세': 5}
X['시술 당시 나이'] = X['시술 당시 나이'].map(age_mapping)
## 알 수 없음을 중간값으로 대체 할 경우 -> "1로 대체됨"
#알 수 없음'을 NaN으로 변환
X['시술 당시 나이'] = X['시술 당시 나이'].replace('알 수 없음', np.nan)

#평균값 계산 (NaN 제외)
mean_age = X['시술 당시 나이'].dropna().astype(float).median()

#NaN을 평균값으로 대체
X['시술 당시 나이'] = X['시술 당시 나이'].astype(float).fillna(mean_age)

In [ ]:
# X 데이터 평균 확인
mean_age

1.0

In [ ]:
# 연령대 컬럼 변환
age_mapping = {'만18-34세': 0, '만35-37세': 1, '만38-39세': 2, '만40-42세': 3, '만43-44세': 4, '만45-50세': 5}
test['시술 당시 나이'] = test['시술 당시 나이'].map(age_mapping)
## 알 수 없음을 중간값으로 대체 할 경우 -> "1로 대체됨"
#알 수 없음'을 NaN으로 변환
test['시술 당시 나이'] = test['시술 당시 나이'].replace('알 수 없음', np.nan)

#평균값 계산 (NaN 제외)
mean_age = test['시술 당시 나이'].dropna().astype(float).median()

#NaN을 평균값으로 대체(X의 평균값인 1으로 값 채움)
test['시술 당시 나이'] = test['시술 당시 나이'].astype(float).fillna(mean_age)

In [ ]:
## 새로운 피쳐 변수 생성(2개)
# '배아 생성 성공률', '배아 이식 성공률'생성
# 새로운 feature 생성 (NaN 및 inf 값 처리 후 정수형 변환)
X['배아 생성 성공률'] = (
    X['총 생성 배아 수'] / X['혼합된 난자 수'].replace(0, 1)
).replace([np.inf, -np.inf], 0).fillna(0).round().astype(int)

X['배아 이식 성공률'] = (
    X['이식된 배아 수'] / X['총 생성 배아 수'].replace(0, 1)
).replace([np.inf, -np.inf], 0).fillna(0).round().astype(int)

test['배아 생성 성공률'] = (
    test['총 생성 배아 수'] / test['혼합된 난자 수'].replace(0, 1)
).replace([np.inf, -np.inf], 0).fillna(0).round().astype(int)

test['배아 이식 성공률'] = (
    test['이식된 배아 수'] / test['총 생성 배아 수'].replace(0, 1)
).replace([np.inf, -np.inf], 0).fillna(0).round().astype(int)

# 결과 확인
print(X[['배아 생성 성공률', '배아 이식 성공률']].describe())

           배아 생성 성공률      배아 이식 성공률
count  256351.000000  256351.000000
mean        0.576120       0.388249
std         0.494173       0.626993
min         0.000000       0.000000
25%         0.000000       0.000000
50%         1.000000       0.000000
75%         1.000000       1.000000
max         1.000000       3.000000
          배아 생성 성공률     배아 이식 성공률
count  90067.000000  90067.000000
mean       0.577737      0.386446
std        0.493923      0.625970
min        0.000000      0.000000
25%        0.000000      0.000000
50%        1.000000      0.000000
75%        1.000000      1.000000
max        1.000000      3.000000


In [ ]:
# 이상치 제거 (IQR을 사용하여 정상 범위 내 값만 유지)
for col in ['배아 생성 성공률', '배아 이식 성공률']:
    Q1 = X[col].quantile(0.25)
    Q3 = X[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    X[col] = np.clip(X[col], lower_bound, upper_bound)
    test[col] = np.clip(test[col], lower_bound, upper_bound)

# 결과 확인
print(X[['배아 생성 성공률', '배아 이식 성공률']].describe())

           배아 생성 성공률      배아 이식 성공률
count  256351.000000  256351.000000
mean        0.576120       0.386880
std         0.494173       0.621813
min         0.000000       0.000000
25%         0.000000       0.000000
50%         1.000000       0.000000
75%         1.000000       1.000000
max         1.000000       2.500000


In [ ]:
# '불임 원인 - 여성 원인' 컬럼만 제거하고 모두 불러오기
categorical_columns = [
    "시술 시기 코드",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [ ]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [ ]:
# OrdinalEncoder: 범주형 데이터를 숫자로 변환
# handle_unknown: 훈련 데이터에 없는 값을 처리하는 방법 설정
# unknown_value: 훈련 데이터에 없는 값을 -1로 처리

ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

범주형데이터를 죄다 문자열로 만들고 바로 ordinalEncoder를 돌려버림

시술 당시 나이의 경우 우리가 한거처럼 손수 옮겨줄 수도 있고

뭐가 더 나은가 생각해보면 좋다고 생각만 해봤음


In [ ]:
# '난자 채취 경과일' 제외하고 모두 불러오기
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일",
    "배아 생성 성공률",
    '배아 이식 성공률'
]

In [ ]:
for col in X_train_encoded.columns:
    print(col, X_train_encoded[col].unique())

시술 시기 코드 [6. 5. 3. 2. 4. 0. 1.]
시술 당시 나이 [0. 5. 1. 2. 3. 4.]
임신 시도 또는 마지막 임신 경과 연수 [nan 10.  6.  7.  8. 12.  9.  3. 16. 11. 15.  5. 13. 17.  4. 14. 19. 20.
 18.  2.  0.  1.]
시술 유형 [1. 0.]
특정 시술 유형 [ 4. 15. 23. 14. 20. 18. 12.  5. 11. 19.  7. 21. 13. 16. 22.  2.  9.  3.
 17. 24.  1. 10.  6.  0.  8.]
배란 자극 여부 [1. 0.]
배란 유도 유형 [0. 3. 2. 1.]
단일 배아 이식 여부 [0. 1. 2.]
착상 전 유전 검사 사용 여부 [1. 0.]
착상 전 유전 진단 사용 여부 [0. 2. 1.]
남성 주 불임 원인 [0. 1.]
남성 부 불임 원인 [0. 1.]
여성 주 불임 원인 [0. 1.]
여성 부 불임 원인 [0. 1.]
부부 주 불임 원인 [0. 1.]
부부 부 불임 원인 [0. 1.]
불명확 불임 원인 [0. 1.]
불임 원인 - 난관 질환 [0. 1.]
불임 원인 - 남성 요인 [1. 0.]
불임 원인 - 배란 장애 [1. 0.]
불임 원인 - 자궁경부 문제 [0. 1.]
불임 원인 - 자궁내막증 [0. 1.]
불임 원인 - 정자 농도 [0. 1.]
불임 원인 - 정자 면역학적 요인 [0. 1.]
불임 원인 - 정자 운동성 [0. 1.]
불임 원인 - 정자 형태 [0. 1.]
배아 생성 주요 이유 [13.  0.  6. 10.  5.  3.  1.  2. 11.  7.  4. 12.  9.  8.]
총 시술 횟수 [0. 1. 2. 6. 3. 4. 5.]
클리닉 내 총 시술 횟수 [0. 1. 2. 6. 3. 5. 4.]
IVF 시술 횟수 [0. 1. 2. 3. 5. 4. 6.]
DI 시술 횟수 [0. 5. 3. 1. 2. 4. 6.]
총 임신 횟수 [0. 1. 2. 3. 4. 5. 6.]
IVF 임신 횟수 [0

In [ ]:
X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [ ]:
for col in X_train_encoded.columns:
    print(col, X_train_encoded[col].unique())

시술 시기 코드 [6. 5. 3. 2. 4. 0. 1.]
시술 당시 나이 [0. 5. 1. 2. 3. 4.]
임신 시도 또는 마지막 임신 경과 연수 [ 0. 10.  6.  7.  8. 12.  9.  3. 16. 11. 15.  5. 13. 17.  4. 14. 19. 20.
 18.  2.  1.]
시술 유형 [1. 0.]
특정 시술 유형 [ 4. 15. 23. 14. 20. 18. 12.  5. 11. 19.  7. 21. 13. 16. 22.  2.  9.  3.
 17. 24.  1. 10.  6.  0.  8.]
배란 자극 여부 [1. 0.]
배란 유도 유형 [0. 3. 2. 1.]
단일 배아 이식 여부 [0. 1. 2.]
착상 전 유전 검사 사용 여부 [1. 0.]
착상 전 유전 진단 사용 여부 [0. 2. 1.]
남성 주 불임 원인 [0. 1.]
남성 부 불임 원인 [0. 1.]
여성 주 불임 원인 [0. 1.]
여성 부 불임 원인 [0. 1.]
부부 주 불임 원인 [0. 1.]
부부 부 불임 원인 [0. 1.]
불명확 불임 원인 [0. 1.]
불임 원인 - 난관 질환 [0. 1.]
불임 원인 - 남성 요인 [1. 0.]
불임 원인 - 배란 장애 [1. 0.]
불임 원인 - 자궁경부 문제 [0. 1.]
불임 원인 - 자궁내막증 [0. 1.]
불임 원인 - 정자 농도 [0. 1.]
불임 원인 - 정자 면역학적 요인 [0. 1.]
불임 원인 - 정자 운동성 [0. 1.]
불임 원인 - 정자 형태 [0. 1.]
배아 생성 주요 이유 [13.  0.  6. 10.  5.  3.  1.  2. 11.  7.  4. 12.  9.  8.]
총 시술 횟수 [0. 1. 2. 6. 3. 4. 5.]
클리닉 내 총 시술 횟수 [0. 1. 2. 6. 3. 5. 4.]
IVF 시술 횟수 [0. 1. 2. 3. 5. 4. 6.]
DI 시술 횟수 [0. 5. 3. 1. 2. 4. 6.]
총 임신 횟수 [0. 1. 2. 3. 4. 5. 6.]
IVF 임신 횟수 [0. 1.

In [ ]:
X_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256351 entries, 0 to 256350
Data columns (total 67 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   시술 시기 코드               256351 non-null  float64
 1   시술 당시 나이               256351 non-null  float64
 2   임신 시도 또는 마지막 임신 경과 연수  256351 non-null  float64
 3   시술 유형                  256351 non-null  float64
 4   특정 시술 유형               256351 non-null  float64
 5   배란 자극 여부               256351 non-null  float64
 6   배란 유도 유형               256351 non-null  float64
 7   단일 배아 이식 여부            256351 non-null  float64
 8   착상 전 유전 검사 사용 여부       256351 non-null  float64
 9   착상 전 유전 진단 사용 여부       256351 non-null  float64
 10  남성 주 불임 원인             256351 non-null  float64
 11  남성 부 불임 원인             256351 non-null  float64
 12  여성 주 불임 원인             256351 non-null  float64
 13  여성 부 불임 원인             256351 non-null  float64
 14  부부 주 불임 원인             256351 non-nu

### Train


In [ ]:
model = ExtraTreesClassifier(random_state=42)

model.fit(X_train_encoded, y)

ExtraTreesClassifier(random_state=42)

In [ ]:
## 피쳐 중요도 계산하기
importances = model.feature_importances_
feature_names = X_train_encoded.columns
feature_importance = pd.Series(importances, index=feature_names).sort_values(ascending=False)
print(feature_importance.head(10))


시술 시기 코드            0.124904
수집된 신선 난자 수         0.085707
배아 이식 경과일           0.070477
총 생성 배아 수           0.064480
시술 당시 나이            0.058823
혼합된 난자 수            0.048171
저장된 배아 수            0.045468
파트너 정자와 혼합된 난자 수    0.044874
클리닉 내 총 시술 횟수       0.039432
총 시술 횟수             0.031844
dtype: float64


In [ ]:
X_train = X_train_encoded
X_test = X_test_encoded

In [ ]:
# Optuna를 통한 하이퍼파라미터 최적화 및 5-fold 교차 검증을 통해 AUC 점수를 계산
## 최적의 하이퍼파라미터 찾기
def objective(trial):
    # 탐색할 파라미터 공간 정의
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'verbosity': -1,
        'boosting_type': 'gbdt',

        # 재현성을 위해 random_state 고정
        'random_state': 42,

        # 모든 CPU 사용
        'n_jobs': -1,

        # Optuna가 탐색할 하이퍼파라미터
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X_train, y)):
        X_fold_train, X_fold_valid = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        y_fold_train, y_fold_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # LightGBM 모델 생성
        model = lgb.LGBMClassifier(**params)

        # Early Stopping을 적용하기 위해 eval_set 지정
        model.fit(
            X_fold_train, y_fold_train,
            eval_set=[(X_fold_valid, y_fold_valid)],
            eval_metric='auc',
            callbacks=[lgb.early_stopping(50)],
        )

        # 예측 확률
        y_proba = model.predict_proba(X_fold_valid)[:, 1]
        fold_score = roc_auc_score(y_fold_valid, y_proba)
        cv_scores.append(fold_score)

    return np.mean(cv_scores)

# Optuna로 최적 파라미터 찾기
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

print("Best parameters:", study.best_params)
print(f"Best CV score: {study.best_value:.4f}")



[I 2025-03-04 13:51:17,213] A new study created in memory with name: no-name-e31bde9d-d39a-49df-9b66-23566549326c


Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[212]	valid_0's auc: 0.733494
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[212]	valid_0's auc: 0.736562
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[212]	valid_0's auc: 0.7354
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[212]	valid_0's auc: 0.734465
Training until validation scores don't improve for 50 rounds


[I 2025-03-04 13:51:37,078] Trial 0 finished with value: 0.7350815400462016 and parameters: {'n_estimators': 212, 'learning_rate': 0.010608047891781807, 'max_depth': 5, 'num_leaves': 84, 'min_child_samples': 24, 'subsample': 0.6904171523180531, 'colsample_bytree': 0.890863096484097, 'reg_alpha': 0.017974450471756644, 'reg_lambda': 0.00010707097015374091}. Best is trial 0 with value: 0.7350815400462016.


Did not meet early stopping. Best iteration is:
[212]	valid_0's auc: 0.735486
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[265]	valid_0's auc: 0.737917
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[289]	valid_0's auc: 0.742331
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[233]	valid_0's auc: 0.739819
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[148]	valid_0's auc: 0.737473
Training until validation scores don't improve for 50 rounds


[I 2025-03-04 13:51:53,727] Trial 1 finished with value: 0.7396008802493089 and parameters: {'n_estimators': 770, 'learning_rate': 0.09194818292381693, 'max_depth': 4, 'num_leaves': 91, 'min_child_samples': 20, 'subsample': 0.7004659473809183, 'colsample_bytree': 0.6763187243026124, 'reg_alpha': 1.5608199933990704e-06, 'reg_lambda': 3.356353597036461e-05}. Best is trial 1 with value: 0.7396008802493089.


...

Early stopping, best iteration is:
[456]	valid_0's auc: 0.740997
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[680]	valid_0's auc: 0.738422
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[517]	valid_0's auc: 0.742543
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[530]	valid_0's auc: 0.740201
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[334]	valid_0's auc: 0.738133
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[541]	valid_0's auc: 0.741271


[I 2025-03-04 14:25:05,983] Trial 98 finished with value: 0.7401141937501521 and parameters: {'n_estimators': 955, 'learning_rate': 0.09221760703179034, 'max_depth': 3, 'num_leaves': 56, 'min_child_samples': 96, 'subsample': 0.8227309762743645, 'colsample_bytree': 0.9612692253224067, 'reg_alpha': 4.077354739110248, 'reg_lambda': 3.008851026597001}. Best is trial 49 with value: 0.7401802531590501.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[602]	valid_0's auc: 0.737971
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[716]	valid_0's auc: 0.742828
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[411]	valid_0's auc: 0.73981
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[357]	valid_0's auc: 0.737922
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[505]	valid_0's auc: 0.740817


[I 2025-03-04 14:25:27,966] Trial 99 finished with value: 0.7398695273320104 and parameters: {'n_estimators': 843, 'learning_rate': 0.08492517688711043, 'max_depth': 3, 'num_leaves': 36, 'min_child_samples': 81, 'subsample': 0.7914217272715485, 'colsample_bytree': 0.9761489166625484, 'reg_alpha': 0.2822384362088526, 'reg_lambda': 5.116422632532542}. Best is trial 49 with value: 0.7401802531590501.


Best parameters: {'n_estimators': 729, 'learning_rate': 0.08581405855203687, 'max_depth': 3, 'num_leaves': 42, 'min_child_samples': 78, 'subsample': 0.8178782142132488, 'colsample_bytree': 0.9800372484455709, 'reg_alpha': 8.249660978537577, 'reg_lambda': 0.3415429882253782}
Best CV score: 0.7402


In [ ]:
# 최적의 하이퍼파라미터 값을 직접 지정 (Optuna를 통해 찾은 값)
best_params = {
        'n_estimators': 862,  # 주어진 최적화된 트리 수
        'learning_rate': 0.08797062915937703,  # 주어진 최적화된 학습률
        'max_depth': 3,  # 주어진 최적화된 최대 깊이
        'num_leaves': 80,  # 주어진 최적화된 리프 수
        'min_child_samples': 82,  # 주어진 최적화된 최소 샘플 수
        'subsample': 0.8971316426273112,  # 주어진 최적화된 샘플링 비율
        'colsample_bytree': 0.7524790699186465,  # 주어진 최적화된 컬럼 샘플링 비율
        'reg_alpha': 9.42278389007915,  # 주어진 최적화된 L1 정규화 계수
        'reg_lambda': 0.00010039295916103793  # 주어진 최적화된 L2 정규화 계수
    }

In [ ]:
# 최적 하이퍼파라미터 확정
best_params = study.best_params
best_params['objective'] = 'binary'
best_params['metric'] = 'auc'
best_params['random_state'] = 42
best_params['n_jobs'] = -1

# 최종 모델 학습 (전체 X_train 사용)
final_model = lgb.LGBMClassifier(**best_params)

# 최종 학습 시에도 early stopping을 고려하고 싶다면
# train set만으로는 어려우므로 별도의 validation set을 떼어두는 방식 권장
final_model.fit(
    X_train, y,
    # eval_set=[(X_val, y_val)],  # 만약 별도의 검증 세트가 있다면
    # early_stopping_rounds=50,
)

LGBMClassifier(colsample_bytree=0.9800372484455709,
               learning_rate=0.08581405855203687, max_depth=3, metric='auc',
               min_child_samples=78, n_estimators=729, n_jobs=-1, num_leaves=42,
               objective='binary', random_state=42, reg_alpha=8.249660978537577,
               reg_lambda=0.3415429882253782, subsample=0.8178782142132488)

### Predict


In [ ]:
# 테스트 데이터 예측
pred_proba = final_model.predict_proba(X_test)[:, 1]

### Submission


In [ ]:
# 제출 파일 생성
sample_submission = pd.read_csv('/content/drive/MyDrive/LG/sample_submission.csv')
sample_submission['probability'] = pred_proba
sample_submission.to_csv('./baseline_submit.csv', index=False)